# Anno 1800 Savegame Visualizer
![Run all cells](imgs/run_all.png) ![Restart kernel](imgs/restart_kernel.png)

In [1]:
import ipywidgets as widgets
with open("imgs/loading-buffering.gif", "rb") as f:
    spinner = widgets.Image(
        value=f.read(),
        format='gif',
        width="5%",
        margin="auto"
    )
    spinner.layout.margin="0 0 0 47.5%"
    display(spinner)

Image(value=b'GIF89a\xf2\x01\xf2\x01\xf7\xa3\x00\xd7\xd7\xda\xd6\xd6\xd9\xd8\xd8\xdb\xd7\xd7\xd9\xf4\xf4\xf4\x…

In [2]:
import importlib
from IPython.display import clear_output

if importlib.util.find_spec("lxml") is None :
    %pip install lxml
    clear_output(wait=True)    
if importlib.util.find_spec("PyQt5") is None :
    %pip install PyQt5
    clear_output(wait=True)
from tools.savegame_visualizer_gui import VisualizerGUI

ui = VisualizerGUI()
try:
    spinner.close()
except:
    pass

ui.display()

Dropdown(index=1, options=('chinese', 'english', 'french', 'german', 'italian', 'japanese', 'korean', 'polish'…

In [3]:
from tools.a7s_model import *
hex_to_int(list(ui.get_island().buildings.values())[0].node.find("./Variation").text)


AttributeError: 'NoneType' object has no attribute 'buildings'

In [65]:
def get_stamp():
    self = ui.get_island()
    options = ui.get_options()
    """

    Args:
        options: See get_layout()

    Returns: XML Tree that can be saved and used as a stamp in Anno

    """
    if len(self.buildings) == 0:
        return None

    e_options = options.get("exclude", {})
    l_options = options.get("label", {})
    c_options = options.get("color", {})
    i_options = options.get("icon", {})

    exclude_blueprints = ("blueprints" in e_options)

    root = ET.Element("Content")

    stamp_path = ET.Element("StampPath")
    stamp_path.text = str(self.session.get_region_guid())
    root.append(stamp_path)

    counts = dict()

    buildings = ET.Element("BuildingInfo")
    tl = self.rectangle[1]
    #print(tl)
    for b in self.buildings.values():
        if b.is_blueprint and exclude_blueprints:
            continue

        btree = ET.Element("None")

        if b.guid in counts:
            counts[b.guid] += 1
        else:
            counts[b.guid] = 1

        def add(name: str, value):
            node = ET.Element(name)
            node.text = str(value)
            btree.append(node)        
        
        pos = b.position[::2] - tl - np.array([0.5,0.5])
        #print(b.position[::2], pos)
        add("Pos", "{} {}".format(pos[0], pos[1]))
        add("Dir", b.direction)
        add("GUID", b.guid)
        if b.variation is not None:
            add("Variation", b.variation)

        if hasattr(b, "modules"):
            add("ComplexOwnerID", self.identifier)

        if hasattr(b, "main_building"):
            add("ComplexOwnerID", getattr(b, "main_building").identifier)

        buildings.append(btree)

    icon_guid = ET.Element("IconGUID")
    icon_guid.text = str(max(counts, key=counts.get))

    root.append(icon_guid)
    root.append(buildings)

    # process streets
    exclude_quay = ("quay" in e_options)

    rect = self.rectangle
    streets = self.session.get_streets()[rect[0][1]:rect[1][1], rect[0][0]:rect[1][0]]
    street_dict = dict()
    count = 0
    for x in range(len(streets)):
        for y in range(len(streets[0])):
            street = A7PARAMS["streets"].get(streets[-y, -x])
            if street is None:
                continue

            if exclude_quay and is_quay(street["id"]):
                continue

            guid = street.get("guid")
            if guid not in street_dict:
                street_dict[guid] = ET.Element("None")

            street_node = ET.Element("None")
            street_node.text = "{} {}".format(-x,-y)
            street_dict[guid].append(street_node)
            count += 1

    street_info = ET.Element("StreetInfo")
    for guid, value in street_dict.items():
        guid_node = ET.Element("None")
        guid_node.text = str(guid)
        street_info.append(guid_node)
        street_info.append(value)
    root.append(street_info)

    street_count = ET.Element("StreetCount")
    street_count.text = str(count)
    root.append(street_count)

    return root

In [75]:
xml = get_stamp()

In [76]:
import lxml
path = ui.get_island().name + ".xml"
with open(path, "wb") as f:
    f.write(lxml.etree.tostring(xml, pretty_print=True))

In [77]:
import subprocess
subprocess.call(["tools/FileDBReader/FileDBReader.exe", "compress", "-i", "tools/FileDBReader/FileFormats/stamp.xml", "-y", "-c", "3", "-f", str(path)])

0

In [74]:
path = "Roads 01.xml"
subprocess.call(["tools/FileDBReader/FileDBReader.exe", "compress", "-i", "tools/FileDBReader/FileFormats/stamp.xml", "-y", "-c", "3", "-f", str(path)])

0

In [ ]:
# Row highlighting in dark mode
# Make trade rout table sortable (or sort by name)